# 1.0 Image Classification
The goal of this tutorial is to familiarize with simple feed-forward neural networks

In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
nclasses = len(class_names)
# summarize loaded dataset
print('Train: X=%s, y=%s' % (train_images.shape, train_labels.shape))
print('Test: X=%s, y=%s' % (test_images.shape, test_labels.shape))
print("unique train labels=%s" % np.unique(train_labels))
print("range values first train img = %s, %s" % (train_images[0].min(), train_images[0].max()))

# preprocessing
train_images = train_images / 255.
test_images = test_images / 255.

In [ ]:
plt.imshow(train_images[0], cmap=plt.cm.binary)

In [ ]:
fig, ax = plt.subplots()
ax.hist(train_labels, bins=np.arange(nclasses + 1), label='train')
ax.hist(test_labels, bins=np.arange(nclasses + 1), label='test')
ax.set_xticks(np.arange(nclasses) + 0.5)
ax.set_xticklabels(class_names, rotation=90)
ax.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(10, 10, figsize=(10,10))
for ax, img, label in zip(axs.flat, train_images, train_labels):
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(img, cmap=plt.cm.binary)
    ax.set_title(class_names[label])
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(nclasses),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
)

model.summary()

In [ ]:
history = model.fit(train_images, train_labels,
                    epochs=10, validation_split=0.33,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 5))
for ax, quantity in zip(axs, ('accuracy', 'loss')):
    ax.plot(history.history[quantity], label='train')
    ax.plot(history.history[f'val_{quantity}'], label='validation')
    ax.legend()
    ax.set_xlabel('epoch', fontsize=15)
    ax.set_ylabel(quantity, fontsize=15)


In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

predictions = probability_model.predict(test_images)
predictions[0]

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(10,10))
for ax, img, prediction, label in zip(axs.flat, test_images, predictions, test_labels):
    ax.set_xticks([])
    ax.set_yticks([])
    ax.grid(False)
    ax.imshow(img, cmap=plt.cm.binary)
    predicted_class_id = np.argmax(prediction)
    ax.set_title("%s (%.0f%%)\n truth:%s" % (class_names[predicted_class_id], prediction[predicted_class_id] * 100, class_names[label]),
                 color='black' if predicted_class_id == label else "red")
    
plt.tight_layout()
plt.show()

In [ ]:
confusion_matrix = tf.math.confusion_matrix(
    test_labels,
    np.argmax(predictions, axis=1),
    num_classes=10,
).numpy()

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(confusion_matrix, xticklabels=class_names, annot=True, yticklabels=class_names, ax=ax, square=True, linewidths=0.1)
ax.set_xlabel('Prediction')
ax.set_ylabel('Truth')

In [ ]:
efficiency = confusion_matrix / np.sum(confusion_matrix, axis=1)  # divide by the truth
purity = (confusion_matrix.T / np.sum(confusion_matrix, axis=0)).T  # divide by the reco

fig, axs = plt.subplots(1, 2, figsize=(15, 7))
sns.heatmap(efficiency * 100, xticklabels=class_names, yticklabels=class_names, ax=axs[0], square=True, linewidths=0.1, annot=True, cmap='Blues')
sns.heatmap(purity * 100, xticklabels=class_names, yticklabels=class_names, ax=axs[1], square=True, linewidths=0.1, annot=True, cmap='Reds')

for ax in axs:
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Truth')
axs[0].set_title('efficiecy = P[prediction|truth]', fontsize=15)
axs[1].set_title('purity = P[truth|prediction]', fontsize=15)
plt.show()

In [ ]:
#Get the predictions for the test data
predicted_classes = np.argmax(predictions, axis=1)
correct = np.nonzero(predicted_classes==test_labels)[0]
incorrect = np.nonzero(predicted_classes!=test_labels)[0]
from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted_classes, target_names=class_names))